<a href="https://colab.research.google.com/github/DuyguA/data_science_scripts/blob/master/fasttext_sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!!pip3 install fasttext

['Collecting fasttext',
 '\x1b[?25l  Downloading https://files.pythonhosted.org/packages/f8/85/e2b368ab6d3528827b147fdb814f8189acc981a4bc2f99ab894650e05c40/fasttext-0.9.2.tar.gz (68kB)',
 '',
 '\x1b[K     |████▊                           | 10kB 22.0MB/s eta 0:00:01',
 '\x1b[K     |█████████▌                      | 20kB 6.7MB/s eta 0:00:01',
 '\x1b[K     |██████████████▎                 | 30kB 7.2MB/s eta 0:00:01',
 '\x1b[K     |███████████████████             | 40kB 8.7MB/s eta 0:00:01',
 '\x1b[K     |███████████████████████▉        | 51kB 7.5MB/s eta 0:00:01',
 '\x1b[K     |████████████████████████████▋   | 61kB 8.4MB/s eta 0:00:01',
 '\x1b[K     |████████████████████████████████| 71kB 4.8MB/s ',
 '\x1b[?25hRequirement already satisfied: pybind11>=2.2 in /usr/local/lib/python3.6/dist-packages (from fasttext) (2.5.0)',
 'Requirement already satisfied: setuptools>=0.7.0 in /usr/local/lib/python3.6/dist-packages (from fasttext) (46.1.3)',
 'Requirement already satisfied: numpy in /usr/lo

In [ ]:
import fasttext

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tr.300.bin.gz

--2020-05-01 10:54:32--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tr.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 2606:4700:10::6816:4a8e, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4506977940 (4.2G) [application/octet-stream]
Saving to: ‘cc.tr.300.bin.gz’

cc.tr.300.bin.gz    100%[===================>]   4.20G  11.0MB/s    in 6m 37s  

2020-05-01 11:01:10 (10.8 MB/s) - ‘cc.tr.300.bin.gz’ saved [4506977940/4506977940]



In [ ]:
!gzip -d cc.tr.300.bin.gz

In [ ]:
m = fasttext.load_model("cc.tr.300.bin")

In [ ]:
dir(m)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_labels',
 '_words',
 'f',
 'get_analogies',
 'get_dimension',
 'get_input_matrix',
 'get_input_vector',
 'get_label_id',
 'get_labels',
 'get_line',
 'get_meter',
 'get_nearest_neighbors',
 'get_output_matrix',
 'get_sentence_vector',
 'get_subword_id',
 'get_subwords',
 'get_word_id',
 'get_word_vector',
 'get_words',
 'is_quantized',
 'labels',
 'predict',
 'quantize',
 'save_model',
 'set_args',
 'set_matrices',
 'test',
 'test_label',
 'words']

In [ ]:
len(y)

7509

In [ ]:
import re

#from emoticon_smiley import replace_smileys

opponents = ["bimcell", "pttcell", "ttnet", "vodafone", "avea", "netcell", "vestelcell", "turk telekom", "turktelekom", "turknet"]
opp_regex = r"@?(" + r"|".join(opponents) + r")(\w+)?"

def replace_ceo(sentence):
  sentence = re.sub(r"(@?Kaan_?Terzio[gğ]lu|\bkaan\b)", "CEO", sentence, flags=re.I)
  return sentence

def replace_opponents(sentence):
  sentence = re.sub(opp_regex, " rakip şirket ", sentence, flags=re.I)
  return sentence

def replace_self_mentions(sentence):
  sentence = re.sub("t.?rkcll?", "turkcell", sentence)
  return sentence

def replace_entities(sentence):
  sentence = replace_ceo(sentence)
  sentence = replace_opponents(sentence)
  #sentence = replace_self_mentions(sentence)
  return sentence

def replace_dots(sentence):
  sentence = sentence.replace(u"\u2026", "...").replace(u"\u2025", "..").replace(u"\u1427", ".")
  return sentence

def replace_brackets(sentence):
  sentence = sentence.replace("}", " ").replace("{", " ").replace("]", " ").replace("[", " ")
  return sentence

def replace_quotation_marks(sentence):
  rlist = [ u"\u2018", u"\u201c", u"\u201d", '"', u"\u0060", u"\u00b4", u"\u2019"]
  for r in rlist:
    sentence = sentence.replace(r, " ")
  return sentence

def replace_url(sentence):
  url_string = r"(http:\/\/www\.|https:\/\/www\.|http:\/\/|https:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,5}(:[0-9]{1,5})?(\/.*)?"
  return re.sub(url_string, " websitesi ", sentence)

def replace_underscore(sentence):
  sentence = re.sub(r"(?<=\s)_(?=\s)", " ", sentence)
  sentence = re.sub(r"(?<=\w)([,:;])", r" \1", sentence)
  sentence = re.sub(r"(?<=[\w\s])([|<>/-])(?=[\w\s])" , " ", sentence)
  return sentence

def process_quotation(sentence):
  sentence = re.sub(r"(?<![.\d])\.", " .", sentence)
  sentence = re.sub(r"\.(?![.\d])", ". ", sentence)
  sentence = sentence.replace("?", " ? ").replace("!", " ! ")
  sentence = re.sub(r"[\s#\@]", " ", sentence)
  return sentence


def preprocess(sentence):
  sentence = sentence.strip()
  sentence = sentence.lower()
  sentence = replace_url(sentence)
  sentence = replace_entities(sentence)
  sentence = replace_dots(sentence)
  sentence = replace_brackets(sentence)
  sentence = replace_quotation_marks(sentence)
  sentence = replace_underscore(sentence)
  sentence = process_quotation(sentence)
  tokens = sentence.strip().split()
  #t = replace_smileys(tokens)
  return  tokens

In [ ]:
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving all_train.csv to all_train.csv


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
sentences = []
y = []

In [ ]:
with open("sarcasm_train.csv", "r", encoding="utf-8") as file:
    for line in file:
        l = line.strip()
        s, res = l.rsplit("\t", 1)
        sentences.append(s)
        y.append(int(res))

In [ ]:
test_sentences = []
test_y = []

with open("test_sarcasm.csv", "r", encoding="utf-8") as file:
    for line in file:
        l = line.strip()
        s, res = l.rsplit("\t", 1)
        test_sentences.append(s)
        test_y.append(int(res))
  

In [ ]:
words = set()
for sentence in sequences:
    for word in sentence:
        words.add(word)

for sentence in test_seqs:
  for word in sentence:
    words.add(word)

n_words = len(words)
print(n_words)

28733


In [ ]:
sequences[1188]
#sentences[1188]

['hmzamrsl', 'medallion', 'ama', 'en', 'iyisi', 'turkcell', '01', '01']

In [ ]:
sequences = [preprocess(sentence) for sentence in sentences]
test_seqs = [preprocess(t_sentence) for t_sentence in test_sentences]  

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D, TimeDistributed
from keras.models import Model
from keras.initializers import Constant
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [ ]:
%tensorflow_version 2.x

In [ ]:
tok = Tokenizer(n_words+1, lower=True)
tok.fit_on_texts(sequences+ test_seqs )
X_word = tok.texts_to_sequences(sequences)
X_word = pad_sequences(X_word, maxlen=MAX_LEN, padding="post")

In [ ]:
test_sequences = tok.texts_to_sequences(test_seqs)
X_test = pad_sequences(test_sequences, MAX_LEN, padding="post")

X_test = np.array(X_test)
test_y = np.array(test_y)

In [ ]:
MAX_LEN =60

In [ ]:
word_index = tok.word_index
embed_size=300
embedding_matrix = np.zeros((n_words+1, embed_size))
for word, i in word_index.items():
    if i >= n_words: continue
    embedding_vector = m.get_word_vector(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
X_word = np.array(X_word)
y = np.array(y)

In [ ]:
word_in = Input(shape=(MAX_LEN,))
x = Embedding(n_words+1, embed_size, embeddings_initializer=Constant(embedding_matrix), input_length=MAX_LEN, trainable=True)(word_in)
x = Bidirectional(LSTM(100, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(100, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=word_in, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_word, y, batch_size=32, epochs=5,  verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
16910/16910 [==============================] - 100s 6ms/step - loss: 0.1004 - accuracy: 0.9672
Epoch 2/5
16910/16910 [==============================] - 99s 6ms/step - loss: 0.0089 - accuracy: 0.9975
Epoch 3/5
16910/16910 [==============================] - 100s 6ms/step - loss: 0.0012 - accuracy: 0.9996
Epoch 4/5
16910/16910 [==============================] - 99s 6ms/step - loss: 2.5855e-04 - accuracy: 0.9999
Epoch 5/5
16910/16910 [==============================] - 99s 6ms/step - loss: 7.0216e-05 - accuracy: 1.0000


In [ ]:
results = model.evaluate(X_test, test_y, batch_size=32)

672/672 [==============================] - 1s 835us/step


In [ ]:
results

[0.05419917544437594, 0.9866071343421936]

In [ ]:
incorrects = np.nonzero(np.round(model.predict(X_test).reshape((-1,))).astype(int) != test_y)

In [ ]:
len(incorrects[0])

9

In [ ]:
for i in incorrects[0]:
  print(test_sentences[i])

turkcel beni bu sefer hangi kampanyayla aldatıcaksın?
turkcell kaplumbağa hızında hizmet veriyor
leylayı bekleyen mecnun olduk ama hala gelmediniz
internetimde çok hızlı donmadan edemiyor
her gün mü geri aranır ne bu sevda sizdeki
gönüllerin birincisi değil küfürlerin birincisisiniz
güzel güzel reklamlar verene kadar güzel güzel işlerinizi rayına soksaydınız daha iyiydi
size müşteri olarak hayatımın hatasını yapmışım
saygısızlık yapmakta daha iyisi yook


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D, TimeDistributed
from keras.models import Model
from keras.initializers import Constant
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, BatchNormalization
from keras.optimizers import Adam

weight_decay = 1e-4
num_filters=64

word_in = Input(shape=(MAX_LEN,))
x = Embedding(n_words+1, embed_size, embeddings_initializer=Constant(embedding_matrix), input_length=MAX_LEN, trainable=True)(word_in)
conv = Conv1D(num_filters, 7, activation='relu', padding='same')(x)
mp1 = MaxPooling1D(2)(conv)
conv1 = Conv1D(num_filters, 7, activation='relu', padding='same')(mp1)
mp2 = GlobalMaxPooling1D()(conv1)
dp1 = Dropout(0.5)(mp2)
dense1  = Dense(32, activation='relu', kernel_regularizer=regularizers.l2(weight_decay))(dp1)
dense2 = Dense(1, activation='sigmoid')(dense1)

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model = Model(input=word_in, outputs=dense2)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


In [ ]:
model.fit(X_word,y, batch_size=32, epochs=10)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
16910/16910 [==============================] - 53s 3ms/step - loss: 0.1305 - accuracy: 0.9488
Epoch 2/10
16910/16910 [==============================] - 52s 3ms/step - loss: 0.0209 - accuracy: 0.9951
Epoch 3/10
16910/16910 [==============================] - 52s 3ms/step - loss: 0.0064 - accuracy: 0.9988
Epoch 4/10
16910/16910 [==============================] - 52s 3ms/step - loss: 0.0045 - accuracy: 0.9993
Epoch 5/10
16910/16910 [==============================] - 51s 3ms/step - loss: 0.0031 - accuracy: 0.9996
Epoch 6/10
16910/16910 [==============================] - 51s 3ms/step - loss: 0.0027 - accuracy: 0.9995
Epoch 7/10
16910/16910 [==============================] - 51s 3ms/step - loss: 0.0035 - accuracy: 0.9993
Epoch 8/10
16910/16910 [==============================] - 51s 3ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 9/10
16910/16910 [==============================] - 51s 3ms/step - loss: 9.4959e-04 - accuracy: 1.0000
Epoch 10/10
16910/16910 [==========================

In [ ]:
results = model.evaluate(X_test, test_y, batch_size=32)

672/672 [==============================] - 0s 224us/step


In [ ]:
results

[0.18152425550146117, 0.9717261791229248]

In [ ]:
incorrects = np.nonzero(np.round(model.predict(X_test).reshape((-1,))).astype(int) != test_y)

In [ ]:
len(incorrects[0])

19

In [ ]:
for i in incorrects[0]:
  print(test_sentences[i], "\t", test_y[i])

turkcel beni bu sefer hangi kampanyayla aldatıcaksın? 	 1
turkcell kaplumbağa hızında hizmet veriyor 	 1
telefon faturam bana mı özel yoksa bir kaç kişinin faturasını da mı ben ödüyorum 	 1
internetimde çok hızlı donmadan edemiyor 	 1
senden operatör saglayıcısı olmaz sadece dert saglayıcı olur 	 1
bu nasıl telefon borcu sanki yeni hat numarası gibi 	 1
evimin toplam gideri telefon faturamdan az geliyor bu ne saçmalık 	 1
şebeke sorunuyla neden bir tek ben ugraşıyorum 	 1
her gün mü geri aranır ne bu sevda sizdeki 	 1
gönüllerin birincisi değil küfürlerin birincisisiniz 	 1
bu faturaya göre benim zengin olmam lazım 	 1
turkcell ve yine usulsüzlük 	 1
modem kurma işini on gündür yapamadılar soran olursa köprü yaptılar 	 1
turkcel dediler geçelim dedik o bize geçirdi 	 1
bıktım artık bu sizin şirketinizin gizli kapaklı oyunlarından 	 1
TurkNet'in Çözümsüz Müşteri Hizmetleri! 	 1
güzel güzel reklamlar verene kadar güzel güzel işlerinizi rayına soksaydınız daha iyiydi 	 1
size müşteri olar

In [ ]:
results2 = model.evaluate(X_sc_test, test_sixclass_y, batch_size=32)

500/500 [==============================] - 0s 250us/step


In [ ]:
results2

[0.012542072430253029, 0.9919999837875366]

In [ ]:
examples = [
            ":)",
            ":))",
            "😚",
            "turkcell :)",
            "seni seviyorum turkcell",
            "turkcell her yerde cekiyor",
            "turkcell her yerde çekiyor",
            "turkcell candır",
            "turkcell candir",
            "🙁",
            "turkcell berbat",
            "turkcell rezilsin",
            "rezilsin turkcell",
            "turkcell 🙁",
            "fizandan geliyor herhalde",
            "faturam fizandan gelicek herhalde",
            "harika müsteri hizmetleri için teşekkür ederim günümü berbat ettiniz"
]

In [ ]:
for ex in examples:
    tt = tok.texts_to_sequences([preprocess(ex)])
    ptt = pad_sequences(tt, MAX_LEN, padding="post")
    pred = model.predict(ptt)
    print(ex)
    print(np.round(pred[0]).astype(int))
    print("---------------")
    print("\n")

:)
[0]
---------------


:))
[0]
---------------


😚
[0]
---------------


turkcell :)
[0]
---------------


seni seviyorum turkcell
[0]
---------------


turkcell her yerde cekiyor
[0]
---------------


turkcell her yerde çekiyor
[0]
---------------


turkcell candır
[0]
---------------


turkcell candir
[0]
---------------


🙁
[0]
---------------


turkcell berbat
[0]
---------------


turkcell rezilsin
[0]
---------------


rezilsin turkcell
[0]
---------------


turkcell 🙁
[0]
---------------


fizandan geliyor herhalde
[1]
---------------


faturam fizandan gelicek herhalde
[1]
---------------


harika müsteri hizmetleri için teşekkür ederim günümü berbat ettiniz
[1]
---------------




### diger model

In [ ]:
from keras.layers import SpatialDropout1D

In [ ]:
word_in = Input(shape=(MAX_LEN,))
emb_word = Embedding(n_words+1, embed_size, embeddings_initializer=Constant(embedding_matrix), input_length=MAX_LEN, trainable=True)(word_in)
x = SpatialDropout1D(0.3)(emb_word)
main_lstm = Bidirectional(LSTM(units=50, return_sequences=False,
                               recurrent_dropout=0.3))(x)
out = Dense(1, activation="sigmoid")(main_lstm)

model = Model(inputs=word_in, outputs=out)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(X_word, y,  batch_size=32, epochs=5, verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
20192/20192 [==============================] - 114s 6ms/step - loss: 0.3529 - accuracy: 0.8439
Epoch 2/5
20192/20192 [==============================] - 113s 6ms/step - loss: 0.0621 - accuracy: 0.9780
Epoch 3/5
20192/20192 [==============================] - 113s 6ms/step - loss: 0.0152 - accuracy: 0.9955
Epoch 4/5
20192/20192 [==============================] - 112s 6ms/step - loss: 0.0094 - accuracy: 0.9969
Epoch 5/5
20192/20192 [==============================] - 111s 6ms/step - loss: 0.0053 - accuracy: 0.9980


In [ ]:
results = model.evaluate(X_test, test_y, batch_size=32)
results

500/500 [==============================] - 0s 704us/step


[1.1021079950332642, 0.8140000104904175]

### Diger model

In [ ]:
word_in = Input(shape=(MAX_LEN,))
emb_word = Embedding(n_words+1, embed_size, embeddings_initializer=Constant(embedding_matrix), input_length=MAX_LEN, trainable=True)(word_in)

main_lstm = Bidirectional(LSTM(units=50, return_sequences=False,
                               recurrent_dropout=0.3))(emb_word)
out = Dense(1, activation="sigmoid")(main_lstm)

model = Model(inputs=word_in, outputs=out)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(X_word, y,batch_size=32, epochs=5, verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
20192/20192 [==============================] - 130s 6ms/step - loss: 0.3183 - accuracy: 0.8613
Epoch 2/5
20192/20192 [==============================] - 126s 6ms/step - loss: 0.0451 - accuracy: 0.9857
Epoch 3/5
20192/20192 [==============================] - 128s 6ms/step - loss: 0.0119 - accuracy: 0.9963
Epoch 4/5
20192/20192 [==============================] - 126s 6ms/step - loss: 0.0072 - accuracy: 0.9976
Epoch 5/5
20192/20192 [==============================] - 126s 6ms/step - loss: 0.0077 - accuracy: 0.9973


In [ ]:
results = model.evaluate(X_test, test_y, batch_size=32)
results

500/500 [==============================] - 0s 680us/step


[1.1701254873275757, 0.8059999942779541]

### diger model

In [ ]:
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, BatchNormalization
from keras.optimizers import Adam

### diger model

In [ ]:
lr = 1e-4
lr_d = 0
units = 128
dr = 0.5

word_in = Input(shape=(MAX_LEN,))
emb_word = Embedding(n_words+1, embed_size, embeddings_initializer=Constant(embedding_matrix), input_length=MAX_LEN, trainable=False)(word_in)
x1 = SpatialDropout1D(dr)(emb_word)
x_gru = Bidirectional(LSTM(units, return_sequences = True))(x1)
x1 = Conv1D(32, kernel_size=3, padding='valid', kernel_initializer='he_uniform')(x_gru)
avg_pool1_gru = GlobalAveragePooling1D()(x1)
max_pool1_gru = GlobalMaxPooling1D()(x1)
    
x3 = Conv1D(32, kernel_size=2, padding='valid', kernel_initializer='he_uniform')(x_gru)
avg_pool3_gru = GlobalAveragePooling1D()(x3)
max_pool3_gru = GlobalMaxPooling1D()(x3)
    
x_lstm = Bidirectional(LSTM(units, return_sequences = True))(x1)
x1 = Conv1D(32, kernel_size=3, padding='valid', kernel_initializer='he_uniform')(x_lstm)
avg_pool1_lstm = GlobalAveragePooling1D()(x1)
max_pool1_lstm = GlobalMaxPooling1D()(x1)
    
x3 = Conv1D(32, kernel_size=2, padding='valid', kernel_initializer='he_uniform')(x_lstm)
avg_pool3_lstm = GlobalAveragePooling1D()(x3)
max_pool3_lstm = GlobalMaxPooling1D()(x3)
    
    
x = concatenate([avg_pool1_gru, max_pool1_gru, avg_pool3_gru, max_pool3_gru, avg_pool1_lstm, max_pool1_lstm, avg_pool3_lstm, max_pool3_lstm])
x = BatchNormalization()(x)
x = Dropout(0.2)(Dense(128,activation='relu') (x))
x = BatchNormalization()(x)
x = Dropout(0.2)(Dense(100,activation='relu') (x))
x = Dense(1, activation = "sigmoid")(x)
model = Model(inputs = word_in, outputs = x)
model.compile(loss = "binary_crossentropy", optimizer = Adam(lr = lr, decay = lr_d), metrics=["accuracy"])

In [ ]:
history = model.fit(X_word, y, batch_size=32, epochs=5, verbose=1)

Epoch 1/5
20192/20192 [==============================] - 144s 7ms/step - loss: 0.6351 - accuracy: 0.6525
Epoch 2/5
20192/20192 [==============================] - 142s 7ms/step - loss: 0.5441 - accuracy: 0.7347
Epoch 3/5
20192/20192 [==============================] - 147s 7ms/step - loss: 0.5187 - accuracy: 0.7553
Epoch 4/5
20192/20192 [==============================] - 143s 7ms/step - loss: 0.5022 - accuracy: 0.7640
Epoch 5/5
20192/20192 [==============================] - 143s 7ms/step - loss: 0.4964 - accuracy: 0.7640


In [ ]:
results = model.evaluate(X_test, test_y, batch_size=32)
results

500/500 [==============================] - 1s 2ms/step


[0.523495970249176, 0.7720000147819519]

### diger model

In [ ]:
word_in = Input(shape=(MAX_LEN,))
emb_word = Embedding(n_words+1, embed_size, embeddings_initializer=Constant(embedding_matrix), input_length=MAX_LEN, trainable=False)(word_in)

dr1 = Dropout(0.2)(emb_word)
dr2 = Conv1D(64, 5, activation='relu')(dr1)
dr3 = MaxPooling1D(pool_size=4)(dr2)
dr4 = Bidirectional(LSTM(300, return_sequences=False))(dr3)
output = Dense(1, activation='sigmoid')(dr4)
model = Model(inputs=word_in, outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_word, y, batch_size=32, epochs=5, verbose=1)

Epoch 1/5
20192/20192 [==============================] - 55s 3ms/step - loss: 0.4602 - accuracy: 0.7832
Epoch 2/5
20192/20192 [==============================] - 53s 3ms/step - loss: 0.3271 - accuracy: 0.8570
Epoch 3/5
20192/20192 [==============================] - 53s 3ms/step - loss: 0.2436 - accuracy: 0.8978
Epoch 4/5
20192/20192 [==============================] - 53s 3ms/step - loss: 0.1708 - accuracy: 0.9314
Epoch 5/5
20192/20192 [==============================] - 53s 3ms/step - loss: 0.1281 - accuracy: 0.9475


In [ ]:
results = model.evaluate(X_test, test_y, batch_size=32)
results

500/500 [==============================] - 0s 933us/step


[0.9650347566604615, 0.7039999961853027]

# CHAR + SUBWORD

In [ ]:
space = " "
nums = "0123456789"
punct = "()+-\"?!,.;:'#@"
alphabet = "aâbcçdefgğhıijklmnoöprsştvwuüyz"
chars = space + nums + punct + alphabet 

In [ ]:
n_chars = len(chars)

In [ ]:
char2idx = {c: i + 2 for i, c in enumerate(chars)}
char2idx["UNK"] = 1
char2idx["PAD"] = 0

In [ ]:
MAX_LEN_CHAR = 25

X_char = []
for sentence in sequences:
    sent_seq = []
    for i in range(MAX_LEN):
        word_seq = []
        for j in range(MAX_LEN_CHAR):
            try:
                word_seq.append(char2idx.get(sentence.lower()[i][j]))
            except:
                word_seq.append(char2idx.get("PAD"))
        sent_seq.append(word_seq)
    X_char.append(np.array(sent_seq))

In [ ]:
word_in = Input(shape=(MAX_LEN,))
emb_word = Embedding(n_words+1, embed_size, embeddings_initializer=Constant(embedding_matrix), input_length=MAX_LEN, trainable=True)(word_in)


char_in = Input(shape=(MAX_LEN, MAX_LEN_CHAR,))
emb_char = Embedding(input_dim=n_chars + 2, output_dim=10,
                           input_length=MAX_LEN_CHAR, mask_zero=True)(char_in)
# character LSTM to get word encodings by characters
char_enc = LSTM(units=20, return_sequences=False, recurrent_dropout=0.3)(emb_char)

# main LSTM
char_enc = GlobalMaxPool1D()(char_enc)

x = concatenate([emb_word, char_enc])
x = SpatialDropout1D(0.3)(x)
main_lstm = Bidirectional(LSTM(units=50, return_sequences=False,
                               recurrent_dropout=0.3))(x)
out = Dense(2, activation="sigmoid")(main_lstm)

model = Model([word_in, char_in], out)

model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001, decay=lr_d), metrics=['accuracy'])

ValueError: ignored

In [ ]:
history = model.fit([X_word,
                     np.array(X_char).reshape((len(X_char), MAX_LEN, MAX_LEN_CHAR))],
                    y,
                    batch_size=32, epochs=10, validation_split=0.1, verbose=1)

Train on 4461 samples, validate on 496 samples
Epoch 1/10
4461/4461 [==============================] - 109s 24ms/step - loss: 0.6914 - acc: 0.7724 - val_loss: 0.6893 - val_acc: 0.8048
Epoch 2/10
4461/4461 [==============================] - 91s 20ms/step - loss: 0.6878 - acc: 0.7729 - val_loss: 0.6854 - val_acc: 0.8048
Epoch 3/10
3008/4461 [===================>..........] - ETA: 29s - loss: 0.6848 - acc: 0.7739

KeyboardInterrupt: ignored

In [ ]:
word_in = Input(shape=(MAX_LEN,))
emb_word = Embedding(n_words+1, embed_size, embeddings_initializer=Constant(embedding_matrix), input_length=MAX_LEN, trainable=True)(word_in)


char_in = Input(shape=(MAX_LEN, MAX_LEN_CHAR,))
emb_char = TimeDistributed(Embedding(input_dim=n_chars + 2, output_dim=10,
                           input_length=MAX_LEN_CHAR, mask_zero=True))(char_in)
# character LSTM to get word encodings by characters
char_enc = TimeDistributed(LSTM(units=20, return_sequences=False,
                                recurrent_dropout=0.3))(emb_char)
# main LSTM
x = concatenate([emb_word, char_enc])
x = SpatialDropout1D(0.3)(x)
main_lstm = Bidirectional(LSTM(units=20, return_sequences=False,
                               recurrent_dropout=0.3))(x)
out = Dense(6, activation="relu")(main_lstm)

model = Model([word_in, char_in], out)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit([X_word,
                     np.array(X_char).reshape((len(X_char), MAX_LEN, MAX_LEN_CHAR))],
                    y,
                    batch_size=32, epochs=10, validation_split=0.1, verbose=1)

Train on 4461 samples, validate on 496 samples
Epoch 1/10
4461/4461 [==============================] - 123s 28ms/step - loss: 3.6607 - acc: 0.7729 - val_loss: 3.1467 - val_acc: 0.8048
Epoch 2/10
4461/4461 [==============================] - 104s 23ms/step - loss: 3.6607 - acc: 0.7729 - val_loss: 3.1467 - val_acc: 0.8048
Epoch 3/10
4461/4461 [==============================] - 104s 23ms/step - loss: 3.6607 - acc: 0.7729 - val_loss: 3.1467 - val_acc: 0.8048
Epoch 4/10
1472/4461 [========>.....................] - ETA: 1:07 - loss: 3.6061 - acc: 0.7763

KeyboardInterrupt: ignored

In [ ]:
for ex in examples:
    tt = tok.texts_to_sequences([ex])
    ptt = pad_sequences(tt, MAX_LEN, padding="post")
    pred = model.predict(ptt)
    print(ex)
    show_pred(pred[0])
    print("---------------")
    print("\n")

In [ ]:
from keras.layers import SpatialDropout1D

word_in = Input(shape=(MAX_LEN,))
emb_word = Embedding(n_words+1, embed_size, embeddings_initializer=Constant(embedding_matrix), input_length=MAX_LEN, trainable=True)(word_in)


char_in = Input(shape=(MAX_LEN, MAX_LEN_CHAR,))
emb_char = TimeDistributed(Embedding(input_dim=n_chars + 2, output_dim=10,
                           input_length=MAX_LEN_CHAR, mask_zero=True))(char_in)
# character LSTM to get word encodings by characters
char_enc = TimeDistributed(LSTM(units=20, return_sequences=False,
                                recurrent_dropout=0.3))(emb_char)
# main LSTM
x = concatenate([emb_word, char_enc])
x = SpatialDropout1D(0.3)(x)
main_lstm = Bidirectional(LSTM(units=50, return_sequences=False,
                               recurrent_dropout=0.3))(x)
out = Dense(1, activation="softmax")(main_lstm)

model = Model([word_in, char_in], out)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit([X_word,
                     np.array(X_char).reshape((len(X_char), MAX_LEN, MAX_LEN_CHAR))],
                    y,
                    batch_size=32, epochs=10, verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
16910/16910 [==============================] - 136s 8ms/step - loss: 13.9269 - accuracy: 0.0917
Epoch 2/10
 9088/16910 [===============>..............] - ETA: 1:02 - loss: 13.8957 - accuracy: 0.0938

KeyboardInterrupt: ignored

In [ ]:
X_test_char = []
for sentence in test_sequences:
    sent_seq = []
    for i in range(MAX_LEN):
        word_seq = []
        for j in range(MAX_LEN_CHAR):
            try:
                word_seq.append(char2idx.get(sentence.lower()[i][j]))
            except:
                word_seq.append(char2idx.get("PAD"))
        sent_seq.append(word_seq)
    X_test_char.append(np.array(sent_seq))
  
results = model.evaluate([X_test, np.array(X_test_char).reshape((len(X_test_char), MAX_LEN, MAX_LEN_CHAR))], test_y, batch_size=32)
results

672/672 [==============================] - 1s 2ms/step


[11.346159253801618, 0.255952388048172]

In [ ]:
!wget vectors.nlpl.eu/repository/11/174.zip

--2020-02-15 21:26:28--  http://vectors.nlpl.eu/repository/11/174.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.225
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 473509943 (452M) [application/zip]
Saving to: ‘174.zip’

174.zip             100%[===================>] 451.57M  20.7MB/s    in 28s     

2020-02-15 21:26:57 (16.1 MB/s) - ‘174.zip’ saved [473509943/473509943]



In [ ]:
!git clone https://github.com/HIT-SCIR/ELMoForManyLangs.git

Cloning into 'ELMoForManyLangs'...
remote: Enumerating objects: 174, done.
remote: Total 174 (delta 0), reused 0 (delta 0), pack-reused 174
Receiving objects: 100% (174/174), 83.75 KiB | 1.74 MiB/s, done.
Resolving deltas: 100% (87/87), done.


In [ ]:
%cd ELMoForManyLangs

/content/ELMoForManyLangs


In [ ]:
!python setup.py install

running install
running bdist_egg
running egg_info
creating elmoformanylangs.egg-info
writing elmoformanylangs.egg-info/PKG-INFO
writing dependency_links to elmoformanylangs.egg-info/dependency_links.txt
writing requirements to elmoformanylangs.egg-info/requires.txt
writing top-level names to elmoformanylangs.egg-info/top_level.txt
writing manifest file 'elmoformanylangs.egg-info/SOURCES.txt'
writing manifest file 'elmoformanylangs.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/elmoformanylangs
copying elmoformanylangs/__init__.py -> build/lib/elmoformanylangs
copying elmoformanylangs/utils.py -> build/lib/elmoformanylangs
copying elmoformanylangs/dataloader.py -> build/lib/elmoformanylangs
copying elmoformanylangs/__main__.py -> build/lib/elmoformanylangs
copying elmoformanylangs/biLM.py -> build/lib/elmoformanylangs
copying elmoformanylangs/frontend.py -> build/lib

In [ ]:
!unzip 174.zip

Archive:  174.zip
  inflating: char.dic                
  inflating: config.json             
  inflating: encoder.pkl             
  inflating: meta.json               
  inflating: README                  
  inflating: token_embedder.pkl      
  inflating: word.dic                


In [ ]:
%cd ..

/content


In [ ]:
!mkdir turc

In [ ]:
!ls

174.zip    config.json	     meta.json	  token_embedder.pkl
174.zip.1  ELMoForManyLangs  README	  turc
char.dic   encoder.pkl	     sample_data  word.dic


In [ ]:
!mv char.dic encoder.pkl config.json meta.json README token_embedder.pkl word.dic turc/

In [ ]:
!ls turc

char.dic			 config.json  meta.json  token_embedder.pkl
cnn_50_100_512_4096_sample.json  encoder.pkl  README	 word.dic


In [ ]:
from elmoformanylangs import Embedder

In [ ]:
e = Embedder('turc/')

2020-02-15 22:02:34,909 INFO: char embedding size: 2518
2020-02-15 22:02:36,844 INFO: word embedding size: 327303
2020-02-15 22:02:41,475 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(327303, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(2518, 50, padding_idx=2515)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 128, kernel_size=(4,), stride=(1,))
      (4): Conv1d(50, 256, kernel_size=(5,), stride=(1,))
      (5): Conv1d(50, 512, kernel_size=(6,), stride=(1,))
      (6): Conv1d(50, 1024, kernel_size=(7,), stride=(1,))
    )
    (highways): Highway(
      (_layers): ModuleList(
        (0): Linear(in_features=2048, out_features=4096, bias=True)
        (1): Linear(in_features=2048, out_fe

In [ ]:
!wget https://raw.githubusercontent.com/HIT-SCIR/ELMoForManyLangs/master/configs/cnn_50_100_512_4096_sample.json

--2020-02-15 21:28:30--  https://raw.githubusercontent.com/HIT-SCIR/ELMoForManyLangs/master/configs/cnn_50_100_512_4096_sample.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 474 [text/plain]
Saving to: ‘cnn_50_100_512_4096_sample.json’

cnn_50_100_512_4096 100%[===================>]     474  --.-KB/s    in 0s      

2020-02-15 21:28:30 (75.0 MB/s) - ‘cnn_50_100_512_4096_sample.json’ saved [474/474]



In [ ]:
!mv cnn_50_100_512_4096_sample.json turc/

In [ ]:
sents = ["dun oraya gittim ama sen ypoktun beki de vardin benki de yoktun", "sen karisma bu ise"]
k = e.sents2elmo(sents)

2020-02-15 21:30:46,582 INFO: 1 batches, avg len: 42.5


In [ ]:
X_sent = e.sents2elmo(sentences[:100])

2020-02-15 22:38:20,998 INFO: 2 batches, avg len: 47.4


In [ ]:
inp = Input(shape = (1,))
emb = Embedding(input_dim=n_words+1, output_dim=20, trainable=True)(inp)
#l_cov1= Conv2D(128, 5, activation='relu')(inp)
#l_pool1 = MaxPooling1D(5)(inp)
#l_flat = Flatten()(inp)
l_dense = Dense(128, activation='relu')(emb)
preds = Dense(2, activation='sigmoid')(l_dense)
model = Model(inputs=inp, outputs=preds)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
z = y[:100]

In [ ]:
from keras.layers import Conv1D , Flatten, Conv2D, MaxPooling1D, Lambda

In [ ]:
history = model.fit(X_sent, z, validation_split=0.1, batch_size=64, epochs=5, verbose=1)

ValueError: ignored

In [ ]:
@tf.function
def ElmoEmbedding(x):
        return tf.convert_to_tensor(e.sents2elmo(x)[0])

In [ ]:
input_text = Input(shape=(120,), dtype=tf.string)
embedding = Lambda(ElmoEmbedding, output_shape=(None, 120, 1024))(input_text)
l_dense = Dense(128, activation='relu')(embedding)
preds = Dense(2, activation='sigmoid')(l_dense)
model = Model(inputs=inp, outputs=preds)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

ValueError: ignored

In [ ]:
import tensorflow as tf